SealNet Detection Pipeline-- deployment
======

SealNet deployment code. This jupyter notebook allows you to make predictions on your own images with the seal detection pipeline. *WARNING* : CNNs used here were trained on panchromatic WorldView-3 imagery so results may not be satisfactory on different imagery / color bands. 

### Load packages

In [2]:
import os

### Call detection pipeline 

The following cell will tile all rasters on *inp_fldr* and generate output shapefiles with s predicted seal points for all architectures listed on *mod_arches*. For a list of supported architectures, training sets , and hyperparameter sets, see: "*utils/model_library.py*" under the main repository.

In [48]:
# select training set, model and hyperparameters
#mod_arches = ['Unet', 'UnetCntWRN', 'UnetCntWRNOccDense']
mod_arches = ['UnetCntWRNOccDense']
t_dir = 'training_set_vanilla'
hyp_set = 'D'

# setup input folder ** change here **
inp_fldr = '/home/bento/GIS_projects/test_imagery'

# loop through architectures
for mod_arch in mod_arches:
    out_fldr = 'test_predictions_{}'.format(mod_arch)

    # loop through images
    inp_imgs = [img for img in os.listdir('{}'.format(inp_fldr)) if img[-4:] == '.tif']
    for img in inp_imgs:
        img_path = '{}/{}'.format(inp_fldr, img)
        out_fldr_img = '{}/{}'.format(out_fldr, img)
        # call pipeline 
        !python predict_raster.py --input_image=$img_path \
                                  --model_architecture=$mod_arch \
                                  --hyperparameter_set=$hyp_set \
                                  --training_set=$t_dir --test_folder=$out_fldr_img \
                                  --stride=0.5


6434 tiles created in 0 minutes and 8.64 seconds
Testing complete in 0h 0m 17s
Total predicted in WV03_20170128160800_104001002876EA00_17JAN28160800-P1BS-501143588010_01_P012_u08rf3031.tif:  0

8523 tiles created in 0 minutes and 10.91 seconds
Testing complete in 0h 0m 21s
Total predicted in WV03_20170122143356_10400100280D8300_17JAN22143356-P1BS-501135119080_01_P004_u08rf3031.tif:  15

7891 tiles created in 0 minutes and 10.74 seconds
Testing complete in 0h 0m 21s
Total predicted in WV03_20170228142819_104001002995B900_17FEB28142819-P1BS-501556010060_01_P001_u08rf3031.tif:  33

9475 tiles created in 0 minutes and 13.72 seconds
Testing complete in 0h 0m 24s
Total predicted in WV03_20170217212037_104001002911B000_17FEB17212037-P1BS-501172848090_01_P005_u08rf3031.tif:  6

6867 tiles created in 0 minutes and 11.04 seconds
Testing complete in 0h 0m 20s
Total predicted in WV03_20170116143631_104001002810BA00_17JAN16143631-P1BS-501135120050_01_P001_u08rf3031.tif:  77

9798 tiles created in 

### Testing predictions

The following cell will merge all predicted shapefiles generated above and check them against a reference shapefile to get precision and recall. For a template on reference shapefiles see "*shape_files/seal-points-consensus.shp*" under the main repository. 

In [13]:
# setup output folder ** change here **
mod_arches = ['Unet', 'UnetCntWRN', 'UnetCntWRNOccDense']
for mod_arch in mod_arches:
    inp_fldr = 'test_predictions_{}'.format(mod_arch)
    out_fldr = '{}/merged_shapefiles'.format(inp_fldr)

    # merge shapefiles
    !python merge_shapefiles.py --input_dir=$inp_fldr --output_dir=$out_fldr

    # test predictions
    shp_prediction = '{}/merged_locations.dbf'.format(out_fldr)
    shp_ground_truth = 'shape_files/seal-points-consensus.dbf'
    output_file = '{}/{}_'.format(inp_fldr, mod_arch)

    !python test_sealnet.py --shp_prediction=$shp_prediction \
                            --shp_ground_truth=$shp_ground_truth \
                            --tolerance_seal=5 \
                            --tolerance_haul=20 \
                            --output_file=$output_file